In [135]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import matplotlib.pyplot as plt
import swan_vis as swan
import yaml
from snakemake.io import expand
import cerberus
import pyranges as pr
import upsetplot

p = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(p)

from scripts.utils import *
from scripts.vcf_utils import *
from scripts.plotting import *
from scripts.sm_utils import *

In [136]:
config = load_config()
od = ''

def proc_cfg(entry, od):
    entry = entry.replace('../../', '')
    entry = od+entry
    return entry

In [154]:
# normal
df = pd.read_csv('../1000g/1000G_metadata.tsv',
    sep='\t', comment='#', header=None)
df = df[[9,10]]
df.columns = ['cell_line_id_1000g', 'population']
df['cell_line_id_1000g_short'] = df['cell_line_id_1000g'].str.split('NA', expand=True)[1]
inds = df.loc[df.cell_line_id_1000g_short.isnull()].index
df.loc[inds, 'cell_line_id_1000g_short'] = df.loc[inds, 'cell_line_id_1000g'].str.split('HG', expand=True)[1]
assert len(df.loc[df.cell_line_id_1000g_short.isnull()]) == 0
df.drop(['cell_line_id_1000g', 'population'], axis=1, inplace=True)

# trios
t_df = pd.read_csv('../1000g/1000G_trios_metadata.tsv',
    sep='\t', comment='#', header=None)
t_df = t_df[[9,10]]
t_df.columns = ['cell_line_id_1000g_trios', 'population']
t_df.head()
t_df['cell_line_id_1000g_trios_short'] = t_df['cell_line_id_1000g_trios'].str.split('NA', expand=True)[1]
inds = t_df.loc[t_df.cell_line_id_1000g_trios_short.isnull()].index
t_df.loc[inds, 'cell_line_id_1000g_trios_short'] = t_df.loc[inds, 'cell_line_id_1000g_trios'].str.split('HG', expand=True)[1]
assert len(t_df.loc[t_df.cell_line_id_1000g_trios_short.isnull()]) == 0
t_df.drop(['cell_line_id_1000g_trios', 'population'], axis=1, inplace=True)

config_df = pd.read_csv('config.tsv', sep='\t')

# need to also get the weird shorthand name from the meta table
# and the 1000g name
meta = load_meta()
meta['hapmap_DNA_ID_short'] = meta['hapmap_DNA_ID'].str.split('NA', expand=True)[1]
config_df = config_df.merge(meta,
                            how='left',
                            on=['cell_line_id', 'population', 'sample'])

# # and also the 1000g trios names
# config_df = config_df.merge(df,
#                   how='left',
#                   left_on='hapmap_DNA_ID_short',
#                   right_on='cell_line_id_1000g_short')

# print(len(config_df.loc[~(config_df.hapmap_DNA_ID.isin(df.cell_line_id_1000g.tolist())), 'cell_line_id'].unique()))
# print(len(config_df.loc[~(config_df.cell_line_id.isin(df.cell_line_id_1000g.tolist())), 'cell_line_id'].unique()))

# print(len(meta.loc[meta.cell_line_id_1000g.isnull()]))
# print(len(meta.loc[meta.cell_line_id_1000g.notnull()]))

# merge w/ 1000g
config_df = config_df.merge(df, 
                  how='left',
                  left_on='hapmap_DNA_ID_short',
                  right_on='cell_line_id_1000g_short')

config_df.loc[(config_df.hapmap_DNA_ID_short.notnull())&\
         (config_df.cell_line_id_1000g_short.isnull())][['sample', 'cell_line_id', 'hapmap_DNA_ID_short',
                                                    'cell_line_id_1000g_short', '1000G']].drop_duplicates()

# TODO remove when matches are all fixed
print(len(config_df.index))
config_df = config_df.loc[config_df.cell_line_id_1000g_short.notnull()]
print(len(config_df.index))

# 43_EU3_GM12812
config_df['lab_sample'] = config_df['lab_number_sample'].astype(str)+'_'+\
                          config_df['lab_sampleid'].astype(str)+'_'+\
                          config_df['cell_line_id'].astype(str)

config_df = config_df[['ref', 'alt', 'chr', 'pos', 'sample']]
config_df.head()

config_df = config_df.melt(
    id_vars=['chr', 'pos', 'sample'], 
    value_vars=['ref', 'alt'], 
    var_name='type', 
    value_name='var')
config_df.head()

813
462


,chr,pos,sample,type,var
0,chr15,88467745,YRI5,ref,C
1,chr17,28468275,YRI5,ref,C
2,chr4,83090803,YRI5,ref,A
3,chr6,31353908,YRI5,ref,A
4,chr6,31353975,YRI5,ref,T


,cell_line_id_1000g_trios_short
0,00405
1,00408
2,00418
3,00420
4,00423


AttributeError: 'DataFrame' object has no attribute 'cell_line_id_1000g_trios_short'

,sample,cell_line_id,hapmap_DNA_ID_short,cell_line_id_1000g_short,1000G
35,YRI7,GM19240,19240,NaN,True
785,YRI3,GM18906,18906,NaN,True


813
462


In [146]:
config_df

,Unnamed: 0,sample,cell_line_id,population,geneid.v,chr,pos,ref,alt,lab_number_sample,...,was_switched_with,relabeled,family_member,family,population_full_name,population _description,extra_info,coriell_link,hapmap_DNA_ID_short,cell_line_id_1000g_short
0,0,YRI5,GM19117,YRI,ENSG00000181991.16,chr15,88467745,C,T,10,...,FALSE,False,Father,Y100,"Yoruba in Ibadan, Nigeria","Yoruba in Ibadan, Nigeria",Empty,https://www.coriell.org/0/Sections/Search/Samp...,19117,19117
1,2,YRI5,GM19117,YRI,ENSG00000171863.15,chr17,28468275,C,T,10,...,FALSE,False,Father,Y100,"Yoruba in Ibadan, Nigeria","Yoruba in Ibadan, Nigeria",Empty,https://www.coriell.org/0/Sections/Search/Samp...,19117,19117
2,4,YRI5,GM19117,YRI,ENSG00000145287.12,chr4,83090803,A,G,10,...,FALSE,False,Father,Y100,"Yoruba in Ibadan, Nigeria","Yoruba in Ibadan, Nigeria",Empty,https://www.coriell.org/0/Sections/Search/Samp...,19117,19117
3,5,YRI5,GM19117,YRI,ENSG00000234745.14,chr6,31353908,A,C,10,...,FALSE,False,Father,Y100,"Yoruba in Ibadan, Nigeria","Yoruba in Ibadan, Nigeria",Empty,https://www.coriell.org/0/Sections/Search/Samp...,19117,19117
4,125,YRI5,GM19117,YRI,ENSG00000234745.14,chr6,31353975,T,G,10,...,FALSE,False,Father,Y100,"Yoruba in Ibadan, Nigeria","Yoruba in Ibadan, Nigeria",Empty,https://www.coriell.org/0/Sections/Search/Samp...,19117,19117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
780,45443,YRI2,GM18489,YRI,ENSG00000196126.12,chr6,32578981,C,T,7,...,FALSE,False,Mother,Y002,"Yoruba in Ibadan, Nigeria","Yoruba in Ibadan, Nigeria",Empty,https://www.coriell.org/0/Sections/Search/Samp...,18489,18489
781,45659,YRI2,GM18489,YRI,ENSG00000231389.8,chr6,33065076,C,T,7,...,FALSE,False,Mother,Y002,"Yoruba in Ibadan, Nigeria","Yoruba in Ibadan, Nigeria",Empty,https://www.coriell.org/0/Sections/Search/Samp...,18489,18489
782,45694,YRI2,GM18489,YRI,ENSG00000231389.8,chr6,33065198,T,C,7,...,FALSE,False,Mother,Y002,"Yoruba in Ibadan, Nigeria","Yoruba in Ibadan, Nigeria",Empty,https://www.coriell.org/0/Sections/Search/Samp...,18489,18489
783,45743,YRI2,GM18489,YRI,ENSG00000281103.5,chr7,38359327,G,T,7,...,FALSE,False,Mother,Y002,"Yoruba in Ibadan, Nigeria","Yoruba in Ibadan, Nigeria",Empty,https://www.coriell.org/0/Sections/Search/Samp...,18489,18489


array(['10_NI5_GM19117', '11_NI6_GM19129', '13_CH1_GM18542',
       '14_CH2_GM18561', '15_CH3_GM18631', '19_KE1_GM19307',
       '20_KE2_GM19328', '22_KE4_GM19390', '23_KE5_GM19446',
       '41_EU1_GM12273', '42_EU2_GM12778', '43_EU3_GM12812',
       '44_EU4_GM12829', '45_EU5_GM12878', '6_NI1_GM18486',
       '7_NI2_GM18489'], dtype=object)

,chr,pos,sample,type,var
0,chr15,88467745,YRI5,ref,C
1,chr17,28468275,YRI5,ref,C
2,chr4,83090803,YRI5,ref,A
3,chr6,31353908,YRI5,ref,A
4,chr6,31353975,YRI5,ref,T


In [149]:
config['lr']['astu_example']['bam']

'../../data/astu_example/{var}_{chr}_{pos}_{sample}.bam'

In [163]:
# expand(config['lr']['astu_example']['bam'],
#        zip,
#        var=config_df['pos'].tolist(),
#        chr=config_df['chr'].tolist(),
#        position=config_df['pos'].tolist(),
#        sample=config_df['sample'].tolist())

## Dev getting the actual reads

In [164]:
import pysam

In [171]:
f = '/Users/fairliereese/Documents/programming/mele_lab/projects/240903_pt/data/03_mapping/pantrx_general_mapping/genomic/12_NI7_GM19240.bam'
bam = pysam.AlignmentFile(f, 'rb')
# {var}_{chr}_{position}
var_pos = 'C_chr6_29942581'
var, ch, pos = var_pos.split('_')
pos = int(pos)

thing = []
thing2 = []
for pileupcolumn in bam.pileup(ch, pos, pos+1):
    thing.append("\n%s = %s" % (pileupcolumn.pos, pileupcolumn.n))
    for pileupread in pileupcolumn.pileups:
        if not pileupread.is_del and not pileupread.is_refskip:
            # query position is None if is_del or is_refskip is set.
            thing2.append('\tbase in read %s = %s' %
                  (pileupread.alignment.query_name,
                   pileupread.alignment.query_sequence[pileupread.query_position]))

bam.close()

In [172]:
len(thing)

335656

In [173]:
len(thing2)

7856787

In [174]:
thing[:5]

['\n29725509 = 1',
 '\n29725510 = 1',
 '\n29725511 = 1',
 '\n29725512 = 1',
 '\n29725513 = 1']